In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np

In [67]:
outt = np.random.rand(3,3,4).astype("uint8")
#print(outt.shape)
w = torch.ones(4,4,3,3)
inp = torch.randn(1,4, 3, 3 )
conv1 = nn.Conv2d(4, 4, 3, 1,1)
conv1.weight = torch.nn.parameter.Parameter(w)
print (conv1.weight.shape)
print(conv1.bias)
#inp.transpose()
out = conv1(inp)
print (out.shape)
out = out.squeeze().transpose(0,2)

outt[0:3,0:3] = out.detach()
print(out)

torch.Size([4, 4, 3, 3])
Parameter containing:
tensor([ 0.0793, -0.0175,  0.1168,  0.0509], requires_grad=True)
torch.Size([1, 4, 3, 3])
tensor([[[ -2.3952,  -2.4920,  -2.3577,  -2.4236],
         [ -4.4351,  -4.5318,  -4.3976,  -4.4634],
         [  1.5878,   1.4910,   1.6253,   1.5595]],

        [[ -8.1680,  -8.2647,  -8.1304,  -8.1963],
         [-10.8311, -10.9279, -10.7936, -10.8594],
         [ -0.1977,  -0.2945,  -0.1602,  -0.2260]],

        [[ -8.4592,  -8.5559,  -8.4217,  -8.4875],
         [-11.2857, -11.3825, -11.2482, -11.3141],
         [ -4.3031,  -4.3999,  -4.2656,  -4.3315]]],
       grad_fn=<TransposeBackward0>)
